# Residual Networks

# 1. ResNet Introduction

### 1.1 Introduction

Very deep neural networks are difficult to train, because of vanishing and exploding gradient types of problems. 

Residual Networks are using <b>skip connections</b> which allows us to take the activation from one layer and suddenly feed it to another layer even much deeper in the neural network. 

ResNets are built out of something called a residual block, let's first describe what that is.


Here is a two layers of neural network:

<center><img src="images/11-CNN/residualblock.PNG"></center>

We have the following steps :

- start off with activaiton $a^{[l]}$
- then we applied the linear operator
- After that you apply the ReLU nonlinearity to get $a^{[l+1]}$
- Then in the next layer you apply the linear step again
- And finally you apply another ReLU to get $a^{[l+2]}$

Following equations :

- $z^{[l+1]} = W^{[l+1]} a^{[l]} + b^{[l+1]}$
- $a^{[l+1]} = g(z^{[l+1]})$
- $z^{[l+2]} = W^{[l+2]} a^{[l+1]} + b^{[l+2]}$
- $a^{[l+2]} = g(z^{[l+2]})$


So in other words, for information from a[l] to flow to a[l+2] it needs to go through all of these steps which is the "main path"


In Residual Net, we are going to take a[l] and apply it further into the neural network.
<center><img src="images/11-CNN/skipconnection.PNG"></center>

So the last equation goes away and we instead have the following output a[l+2]:
- $a^{[l+2]} = g(z^{[l+2]} + a^{[l]})$

<b>Skip connection</b> refers to the activation a[l] just skipping over a layer in order to process information deeper into the neural network.

Using residual blocks allows us to train much deeper neural networks. A way to build a ResNet is by taking many of these Residual blocks, and stacking them together to form a deep network.

<center><img src="images/11-CNN/residualblock2.PNG"></br><u><em>Figure: Residual Block</em></u>
</center>


Difference between a plain network and a Resnet:

<center><img src="images/11-CNN/plainnetwork.PNG"></br><u><em>Figure: Plain Network</em></u>
</center>
<br>
<center><img src="images/11-CNN/resnet.PNG"></br><u><em>Figure: Residual Network</em></u>
</center>

Empirically, you find that as you increase the number of layers, the training error will tend to decreaser after a while but then they'll tend to go back up. And in theory as you make a neural network deeper it should only do better and better on the training set. So having a deeper network should only help. But in practice or in reality, having <b>a plain network</b> that is very deep means that all your optimization algorithm just has a much harder time training. And so in reality your training error gets worse if you pick a network that's too deep.<br>
But what happens with ResNet is that even as the number of layers gets deeper, you can have the performance of the training error king of keep going down. 

<center><img src="images/11-CNN/trainingerror.PNG">
</center>

### 1.2 Why ResNet Works?